# Auffällige Mehrfachwerte je Provider
Einige Felder können je `dataprovider_id` mehrere unterschiedliche Werte haben (z.B. durch Datenmischung).
Dafür wird ein Solr JSON-Facet genutzt und anschließend client-seitig auf `nvals > 1` sowie 32-stellige IDs gefiltert.
Hinweis: `unique(...)` ist in Solr oft ein schneller, näherungsweiser Distinct-Count (für das Finden von Kandidaten meist ausreichend).

In [1]:
import requests
import pandas as pd
import json

In [2]:
# Organization-Daten (ddb-institution) laden -> DataFrame
url_org = "https://api.deutsche-digitale-bibliothek.de/2/search/index/organization/select"
params_org = {
    "q": "type:ddb-institution OR type:ddb-aggregator",
    "fl": "id,label",
    "rows": 10000,
    "wt": "json",
}

resp_org = requests.get(url_org, params=params_org, timeout=300)
resp_org.raise_for_status()
data_org = resp_org.json()

docs = data_org.get("response", {}).get("docs", [])

def _as_list(value):
    if value is None:
        return []
    # Solr liefert bei *_fct i.d.R. Listen; zur Sicherheit wird auch ein Scalar zu einer Liste normalisiert
    return value if isinstance(value, list) else [value]

df_org = pd.DataFrame({
    "id": [d.get("id") for d in docs],
    "label": [_as_list(d.get("label")) for d in docs],
})

df_org

,id,label
0,5IMBCSZN7ZVL77HVH4GKX3GXNPD6X2NO,[Deutsche Fotothek]
1,DWIVLJW5JBO7X4XRO2Y7PCGZSHLTUS6F,[Digitales Kunst- und Kulturarchiv Düsseldorf ...
2,7GAIUS5GGMSBZWXPPPNTXAQPG5UPXKUA,[Rheinische Friedrich-Wilhelms-Universität Bonn]
3,X5L26OPZUGHRAXCAENZ4YSIABDSQBDCG,[Georg-August-Universität Göttingen]
4,IWHIY724L2AQF33GY3GGGIMUMQB5IJ3L,[Universität Hamburg]
...,...,...
4944,QGCRF5ETJ7S746OUN3EWT76HLCI2EJNF,[Stadtarchiv Holzminden]
4945,5XW2AKKSDPK3RF5NXIFRV4WOYOVF5UFW,[Kreisarchiv des Landkreises Verden]
4946,6RZWY2PNOT35GHAV3V5A3WOOMMFLMCUA,[Deutsches Adelsarchiv]
4947,DPKPCTEIZ5OAC4Z45H54PK3ABSCJTVZK,[Stadtarchiv Friedrichroda]


In [3]:
url = "https://api.deutsche-digitale-bibliothek.de/2/search/index/search/select"
params = {
    "q": "*:*",
    "rows": 0,
    "wt": "json",
    "json.facet": json.dumps({
        "providers": {
            "type": "terms",
            "field": "dataprovider_id",
            "limit": -1,
            "mincount": 1,
            "facet": {
                # distinct count (i.d.R. HLL/approx, dafür schnell)
                "nvals": "unique(dataprovider_fct)"
            },
            "sort": "nvals desc"
        }
    })
}

j = requests.get(url, params=params, timeout=300).json()
buckets = j["facets"]["providers"]["buckets"]

# Nur auffällige Fälle
auffaellig = [b for b in buckets if b.get("nvals", 0) > 1]

# -> DataFrame
df_auffaellig = pd.DataFrame([
    {
        "dataprovider_id": str(b.get("val", "")),
        "dataprovider_fct_nvalues": int(b.get("nvals", 0) or 0),
    }
    for b in auffaellig
])

# nur 32-stellige IDs behalten
df_auffaellig = df_auffaellig[df_auffaellig["dataprovider_id"].str.len() == 32].copy()

# Label aus df_org anreichern (falls df_org existiert)
df_auffaellig = (
    df_auffaellig
    .merge(df_org[["id", "label"]], left_on="dataprovider_id", right_on="id", how="left")
    .drop(columns=["id"])
)

df_auffaellig

,dataprovider_id,dataprovider_fct_nvalues,label
0,7A5GWWGTIAUDXLN6JNTFKKZUM27GCGGI,2,[Staatliche Kunstsammlungen Dresden. GRASSI Mu...
1,ZCXCMB7WXARQK27QKZY6ZQSH23D4YEOA,2,[Queer*Feministische Bibliothek und Archiv LIE...


## Variante: Mehrfachwerte in `sector_fct`
Gleiche Logik wie oben, aber für `sector_fct` (Sektor-Klassifikation).
Ergebnis: `df_auffaellig`, angereichert mit `label` aus `df_join`.

In [4]:
url = "https://api.deutsche-digitale-bibliothek.de/2/search/index/search/select"
params = {
    "q": "*:*",
    "rows": 0,
    "wt": "json",
    "json.facet": json.dumps({
        "providers": {
            "type": "terms",
            "field": "dataprovider_id",
            "limit": -1,
            "mincount": 1,
            "facet": {
                # distinct count (i.d.R. HLL/approx, dafür schnell)
                "nvals": "unique(sector_fct)"
            },
            "sort": "nvals desc"
        }
    })
}

j = requests.get(url, params=params, timeout=300).json()
buckets = j["facets"]["providers"]["buckets"]

# Nur auffällige Fälle
auffaellig = [b for b in buckets if b.get("nvals", 0) > 1]

# -> DataFrame
df_auffaellig = pd.DataFrame([
    {
        "dataprovider_id": str(b.get("val", "")),
        "sector_fct_nvalues": int(b.get("nvals", 0) or 0),
    }
    for b in auffaellig
])

# nur 32-stellige IDs behalten
df_auffaellig = df_auffaellig[df_auffaellig["dataprovider_id"].str.len() == 32].copy()

# Label aus df_org anreichern (falls df_org existiert)
df_auffaellig = (
    df_auffaellig
    .merge(df_org[["id", "label"]], left_on="dataprovider_id", right_on="id", how="left")
    .drop(columns=["id"])
)

df_auffaellig

,dataprovider_id,sector_fct_nvalues,label
0,A52WOLFHOQ4JDAPIGD7O7XJTP6N7FZ5E,2,[Stiftung Berliner Mauer]
